# Observing Blocks

In [1]:
from keckODL.target import Target, TargetList
from keckODL.kcwi import KCWIblueDetectorConfig, KCWIredDetectorConfig
from keckODL.kcwi import KCWIConfig
from keckODL.offset import Stare, StarSkyStar, SkyStar
from keckODL.block import ObservingBlock, ObservingBlockList
from keckODL import upload_to_DB

from astropy import units as u

# KCWI + KCRM Example

In [2]:
ngc1333 = Target('NGC1333', rotmode='PA', PA=22.5,
                 objecttype='science',
                 acquisition='blind')

In [3]:
# Let's set up detector configurations for the red and blue arm
# For the blue side we'll take a single 1800s exposure
# For the red side, we'll do two 900s exposures
kcwib_1800s = KCWIblueDetectorConfig(exptime=1800, nexp=1, binning='2x2', readoutmode=0, ampmode=9)
kcwir_1800s = KCWIredDetectorConfig(exptime=900, nexp=2, binning='2x2', readoutmode=0, ampmode=9)
kcwib_1800s.write('detector_KCWIb_1800s.txt')
kcwir_1800s.write('detector_KCWIr_1800s.txt')

In [4]:
# How long will the blue side exposure take with overheads?
kcwib_1800s.estimate_clock_time()

1853

In [5]:
# How long will the red side exposure take with overheads?
kcwir_1800s.estimate_clock_time()

1800

In [6]:
# Let's ask the red side to adjust the exposure time to 
# match the time it takes for the blue side to finish
kcwir_1800s.match_time(kcwib_1800s.estimate_clock_time())
kcwir_1800s.exptime

926.5

In [7]:
# Now the red side will finish at the same time the blue side does
kcwir_1800s.estimate_clock_time()

1853.0

In [8]:
med_slicer_b4800_r6500 = KCWIConfig(slicer='medium', bluegrating='BH3', bluecwave=4800,
                                    redgrating='RH3', redcwave=6563)
med_slicer_b4800_r6500.write('instrument_KCWI_medslicer_red_blue.txt')
med_slicer_b4800_r6500

medium BH3 4800 A

In [9]:
skyoffset = StarSkyStar(dx=60*u.arcsec, dy=30*u.arcsec, repeat=3)
skyoffset.write('offset_customStarSkyStar.txt')

In [10]:
OB1 = ObservingBlock(target=ngc1333, pattern=skyoffset,
                     instconfig=med_slicer_b4800_r6500,
                     detconfig=(kcwib_1800s, kcwir_1800s))

In [11]:
OBlist = ObservingBlockList([OB1])
OBlist

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
NGC1333        |StarSkyStar (60 30) x3|medium BH3 4800 A                            |(KCWI 1800s x1, KCWI 926s x2)       

In [12]:
upload_to_DB([skyoffset, ngc1333, med_slicer_b4800_r6500]) # Will fail if not inside Keck network

Unable to connect to Keck DB


In [13]:
OBlist.estimate_time()

Shutter Open Time: 16677.0 s (4.6 hrs)
Wall Clock Time: 16677 s (4.6 hrs)


In [14]:
cals_I_need = OBlist.cals()
cals_I_need

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
None           |Stare x1              |medium BH3 4800 A arclamp=CONT calobj=MedBarsA|[KCWI 6s x1]                        
None           |Stare x1              |medium BH3 4800 A arclamp=FEAR calobj=FlatA  |[KCWI 30s x1]                       
None           |Stare x1              |medium BH3 4800 A arclamp=THAR calobj=FlatA  |[KCWI 45s x1]                       
None           |Stare x1              |medium BH3 4800 A arclamp=CONT calobj=FlatA  |[KCWI 6s x1]                        
None           |Stare x7              |medium BH3 4800 A bias                       |[KCWI 0s (Dark) x1]                 
DomeFlats      |Stare x3              |medium BH3 4800 A domeflatlamp=True          |[KCWI 100s x1]                      

In [15]:
cals_I_need.estimate_time()

Shutter Open Time: 387 s (0.1 hrs)
Wall Clock Time: 2767 s (0.8 hrs)


# NIRES SCAM + SVC Example

In [16]:
from keckODL.target import Target, TargetList
from keckODL.nires import NIRESConfig, NIRESSpecConfig, NIRESScamConfig, ABBA
from keckODL.offset import Stare
from keckODL.block import ObservingBlock, ObservingBlockList
from astropy import units as u

In [17]:
v1647Ori = Target('V1647 Ori', rotmode='PA', PA=22.5,
                  objecttype='science',
                  acquisition='blind')

In [18]:
spec_300s = NIRESSpecConfig(exptime=300, readoutmode='MCDS32', coadds=1)
scam_10s = NIRESScamConfig(exptime=10, readoutmode='CDS', coadds=1, nexp=6)
spec_300s.write('detector_NIRESspec_300s.txt')
scam_10s.write('detector_NIRESscam_6x10s.txt')
print(spec_300s)
print(scam_10s)

NIRES Spec 300s (MCDS32, 1 coadds) x1
NIRES SCAM 10s (CDS, 1 coadds) x6


In [19]:
# instrument config for NIRES is simple
nires_300s = NIRESConfig()
nires_300s.write('instrument_NIRES_300s.txt')
nires_300s

NIRES Instrument Config

In [20]:
abba5 = ABBA(offset=10*u.arcsec, repeat=5)
print(abba5)

ABBA (10.00 arcsec) x5


In [21]:
myOBs = ObservingBlockList([ObservingBlock(target=v1647Ori, pattern=abba5, instconfig=nires_300s, detconfig=[spec_300s, scam_10s]),
                           ])
print(myOBs)

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
V1647 Ori      |ABBA (10.00 arcsec) x5|NIRES Instrument Config                      |[NIRES Spec 300s (MCDS32, 1 coadds) x1, NIRES SCAM 10s (CDS, 1 coadds) x6]


In [22]:
cals_I_need = myOBs.cals()
print(cals_I_need)

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
None           |Stare x3              |NIRES Instrument Config arclamp              |[NIRES Spec 120s (CDS, 1 coadds) x1]
DomeFlats      |Stare x9              |NIRES Instrument Config domelamp=on          |[NIRES Spec 100s (CDS, 1 coadds) x1]


## MOSFIRE Example

In [23]:
from keckODL.target import Target, TargetList
from keckODL.mosfire import MOSFIREConfig, MOSFIREDetectorConfig, long2pos, ABBA
from keckODL.offset import Stare
from keckODL.block import ObservingBlock, ObservingBlockList
from astropy import units as u

In [24]:
v1647Ori = Target('V1647 Ori', rotmode='PA', PA=22.5,
                  objecttype='science',
                  acquisition='blind')

In [25]:
# For MOSFIRE we leave nexp=1 in the detector configs because we only
# want one exposure at each point in the dither pattern.
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
mosfire_180s.write('detector_MOSFIRE_180s.txt')
mosfire_120s.write('detector_MOSFIRE_120s.txt')
print(mosfire_180s)
print(mosfire_120s)

MOSFIRE 180s (MCDS16, 1 coadds) x1
MOSFIRE 120s (MCDS16, 1 coadds) x1


In [26]:
# instrument configs
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
mosfire_Y.write('instrument_MOSFIRE_Y.txt')
mosfire_J.write('instrument_MOSFIRE_J.txt')
mosfire_H.write('instrument_MOSFIRE_H.txt')
mosfire_K.write('instrument_MOSFIRE_K.txt')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

my_MOS_mask Y-spectroscopy
my_MOS_mask J-spectroscopy
my_MOS_mask H-spectroscopy
my_MOS_mask K-spectroscopy


In [27]:
abba5 = ABBA(offset=1.25*u.arcsec, repeat=5)
print(abba5)
abba8 = ABBA(offset=1.25*u.arcsec, repeat=8)
print(abba8)

ABBA (1.25 arcsec) x5
ABBA (1.25 arcsec) x8


In [28]:
abba5

Frame: MOSFIRE Slit (INSTXOFF INSTYOFF)
Repeats: 5
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
  +0.0|  +1.2|    +0.0|       A|  True
  +0.0|  -1.2|    +0.0|       B|  True
  +0.0|  -1.2|    +0.0|       B|  True
  +0.0|  +1.2|    +0.0|       A|  True

In [29]:
for i,offset in enumerate(abba5):
    print(f'Executing offset: {i}')
    offset.execute()

Executing offset: 0
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(1.25, rel2base=t)
Executing offset: 1
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(-1.25, rel2base=t)
Executing offset: 2
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(-1.25, rel2base=t)
Executing offset: 3
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(1.25, rel2base=t)


In [30]:
myOBs = ObservingBlockList([ObservingBlock(target=v1647Ori, pattern=abba5, instconfig=mosfire_Y, detconfig=mosfire_180s),
                            ObservingBlock(target=v1647Ori, pattern=abba8, instconfig=mosfire_J, detconfig=mosfire_120s),
                            ObservingBlock(target=v1647Ori, pattern=abba8, instconfig=mosfire_H, detconfig=mosfire_120s),
                            ObservingBlock(target=v1647Ori, pattern=abba5, instconfig=mosfire_K, detconfig=mosfire_180s),
                           ])
print(myOBs)

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
V1647 Ori      |ABBA (1.25 arcsec) x5 |my_MOS_mask Y-spectroscopy                   |[MOSFIRE 180s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x8 |my_MOS_mask J-spectroscopy                   |[MOSFIRE 120s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x8 |my_MOS_mask H-spectroscopy                   |[MOSFIRE 120s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x5 |my_MOS_mask K-spectroscopy                   |[MOSFIRE 180s (MCDS16, 1 coadds) x1]


In [31]:
myOBs.estimate_time()

Shutter Open Time: 14880 s (4.1 hrs)
Wall Clock Time: 14880 s (4.1 hrs)


In [32]:
cals_I_need = myOBs.cals()
print(cals_I_need)

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
DomeFlats      |Stare x7              |my_MOS_mask Y-spectroscopy domelamp=on       |[MOSFIRE 17s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |my_MOS_mask H-spectroscopy domelamp=on       |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |my_MOS_mask J-spectroscopy domelamp=on       |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |my_MOS_mask K-spectroscopy domelamp=on       |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |my_MOS_mask K-spectroscopy domelamp=off      |[MOSFIRE 11s (CDS, 1 coadds) x1]    
None           |Stare x2              |my_MOS_mask K-spectroscopy arclamp=Ne        |[MOSFIRE 1s (CDS, 1 coadds) x1]     
None           |Stare x2

In [33]:
cals_I_need.estimate_time()

Shutter Open Time: 431 s (0.1 hrs)
Wall Clock Time: 431 s (0.1 hrs)


## MOSFIRE Example 2

In [34]:
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_3x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_3x0.7')
singleobj_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='long2pos')
singleobj_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='long2pos')
print(singleobj_Y)
print(singleobj_J)
print(singleobj_H)
print(singleobj_K)

longslit_3x0.7 Y-spectroscopy
longslit_3x0.7 J-spectroscopy
long2pos H-spectroscopy
long2pos K-spectroscopy


In [35]:
abba2 = ABBA(offset=1.25*u.arcsec, repeat=2)
print(abba2)
long2pos_pattern = long2pos(repeat=2)
print(long2pos_pattern)

ABBA (1.25 arcsec) x2
long2pos x2


In [36]:
long2pos_pattern

Frame: MOSFIRE Detector (INSTXOFF INSTYOFF)
Repeats: 2
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
 +45.0| -23.0|    +0.0|       A|  True
 +45.0|  -9.0|    +0.0|       B|  True
 -45.0|  +9.0|    +0.0|       A|  True
 -45.0| +23.0|    +0.0|       B|  True

In [37]:
long2pos_pattern[0].execute()

'INSTXOFF'.write(45.0, rel2base=t)
'INSTYOFF'.write(-23.0, rel2base=t)


In [38]:
myOBs2 = ObservingBlockList([ObservingBlock(target=v1647Ori, detconfig=mosfire_180s, pattern=abba2, instconfig=singleobj_Y),
                             ObservingBlock(target=v1647Ori, detconfig=mosfire_120s, pattern=abba2, instconfig=singleobj_J),
                             ObservingBlock(target=v1647Ori, detconfig=mosfire_120s, pattern=long2pos_pattern, instconfig=singleobj_H),
                             ObservingBlock(target=v1647Ori, detconfig=mosfire_180s, pattern=long2pos_pattern, instconfig=singleobj_K),
                            ])
print(myOBs2)

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
V1647 Ori      |ABBA (1.25 arcsec) x2 |longslit_3x0.7 Y-spectroscopy                |[MOSFIRE 180s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x2 |longslit_3x0.7 J-spectroscopy                |[MOSFIRE 120s (MCDS16, 1 coadds) x1]
V1647 Ori      |long2pos x2           |long2pos H-spectroscopy                      |[MOSFIRE 120s (MCDS16, 1 coadds) x1]
V1647 Ori      |long2pos x2           |long2pos K-spectroscopy                      |[MOSFIRE 180s (MCDS16, 1 coadds) x1]


In [39]:
myOBs2.estimate_time()

Shutter Open Time: 4800 s (1.3 hrs)
Wall Clock Time: 4800 s (1.3 hrs)


In [40]:
cals_I_need = singleobj_Y.cals()
cals_I_need.extend(singleobj_J.cals())
cals_I_need.extend(singleobj_H.cals())
cals_I_need.extend(singleobj_K.cals())
print(cals_I_need)

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
DomeFlats      |Stare x7              |longslit_3x0.7 Y-spectroscopy domelamp=on    |[MOSFIRE 17s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |longslit_3x0.7 J-spectroscopy domelamp=on    |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |long2pos H-spectroscopy domelamp=on          |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |long2pos K-spectroscopy domelamp=on          |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |long2pos K-spectroscopy domelamp=off         |[MOSFIRE 11s (CDS, 1 coadds) x1]    
None           |Stare x2              |long2pos K-spectroscopy arclamp=Ne           |[MOSFIRE 1s (CDS, 1 coadds) x1]     
None           |Stare x2

In [41]:
cals_I_need.estimate_time()

Shutter Open Time: 431 s (0.1 hrs)
Wall Clock Time: 431 s (0.1 hrs)


# MOSFIRE Night Example

In [42]:
myOBs3 = ObservingBlockList([ObservingBlock(target=ngc1333, pattern=abba2, instconfig=singleobj_Y),
                             ObservingBlock(target=ngc1333, pattern=abba2, instconfig=singleobj_J),
                             ObservingBlock(target=ngc1333, pattern=long2pos_pattern, instconfig=singleobj_H),
                             ObservingBlock(target=ngc1333, pattern=long2pos_pattern, instconfig=singleobj_K),
                            ])
print(myOBs3)

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
NGC1333        |ABBA (1.25 arcsec) x2 |longslit_3x0.7 Y-spectroscopy                |[None]                              
NGC1333        |ABBA (1.25 arcsec) x2 |longslit_3x0.7 J-spectroscopy                |[None]                              
NGC1333        |long2pos x2           |long2pos H-spectroscopy                      |[None]                              
NGC1333        |long2pos x2           |long2pos K-spectroscopy                      |[None]                              


In [43]:
myOBs.extend(myOBs2)
myOBs.extend(myOBs3)

In [44]:
myOBs

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
V1647 Ori      |ABBA (1.25 arcsec) x5 |my_MOS_mask Y-spectroscopy                   |[MOSFIRE 180s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x8 |my_MOS_mask J-spectroscopy                   |[MOSFIRE 120s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x8 |my_MOS_mask H-spectroscopy                   |[MOSFIRE 120s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x5 |my_MOS_mask K-spectroscopy                   |[MOSFIRE 180s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x2 |longslit_3x0.7 Y-spectroscopy                |[MOSFIRE 180s (MCDS16, 1 coadds) x1]
V1647 Ori      |ABBA (1.25 arcsec) x2 |longslit_3x0.7 J-spectroscopy                |[MOSFIRE 120s (MCDS16, 1 coadds) x1]
V1647 Ori      |long2pos

In [45]:
myOBs.cals()

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
DomeFlats      |Stare x7              |my_MOS_mask H-spectroscopy domelamp=on       |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |my_MOS_mask K-spectroscopy domelamp=on       |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7              |my_MOS_mask K-spectroscopy domelamp=off      |[MOSFIRE 11s (CDS, 1 coadds) x1]    
None           |Stare x2              |my_MOS_mask K-spectroscopy arclamp=Ne        |[MOSFIRE 1s (CDS, 1 coadds) x1]     
None           |Stare x2              |my_MOS_mask K-spectroscopy arclamp=Ar        |[MOSFIRE 1s (CDS, 1 coadds) x1]     
DomeFlats      |Stare x7              |long2pos H-spectroscopy domelamp=on          |[MOSFIRE 11s (CDS, 1 coadds) x1]    
DomeFlats      |Stare x7

# Build Complex MOSFIRE Example

Observer wants to:
- slew to an offset star
- align on the offset star
- offset to target
- image target though the alignment mask
- if target is visible, configure science mask and take spectra, if not, go to next target

In [46]:
from keckODL.target import Target, TargetList
from keckODL.mosfire import MOSFIREConfig, MOSFIREDetectorConfig, long2pos, ABBA
from keckODL.offset import Stare, StarSkyStar, SkyStar
from keckODL.offset import Stare
from keckODL.block import ObservingBlock, ObservingBlockList
from astropy import units as u

In [47]:
# Build target
t1 = Target(name='M42', RA=82.5, Dec=-5, frame='icrs',
            PMRA=10, PMDec=-10, epoch=1975.5,
            rotmode='PA', PA=90,
            objecttype='science',
            RAOffset=8.4, DecOffset=-4.5,
            acquisition='mask align + offset',
            mag={'V': 4.3, 'K': 6.7},
           )
# Have second version of the target with acquisition none
from copy import deepcopy
t2 = deepcopy(t1)
t2.acquisition = 'none'

In [48]:
mosfire_30sx3 = MOSFIREDetectorConfig(exptime=30, readoutmode='CDS', coadds=3)
mosfire_10sx9 = MOSFIREDetectorConfig(exptime=10, readoutmode='CDS', coadds=9)

mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)

In [49]:
skystar = SkyStar(dx=10*u.arcsec, dy=10*u.arcsec)
abba3 = ABBA(offset=1.25*u.arcsec, repeat=3)

In [50]:
skystar

Frame: SkyFrame (RAOFF DECOFF)
Repeats: 1
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
 +10.0| +10.0|    +0.0|     sky| False
  +0.0|  +0.0|    +0.0|    star|  True

In [51]:
# Build explicit align version of the mask so it can be sent as unique science mask
singleobj_J_align = MOSFIREConfig(mode='imaging', filter='J', mask='longslit_align_46x0.7')
# Build normal spectroscopy masks
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_46x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_46x0.7')


In [52]:
myOBs3 = ObservingBlockList([ObservingBlock(target=t1, pattern=skystar, instconfig=singleobj_J_align, detconfig=mosfire_10sx9),
                             ObservingBlock(target=t2, pattern=abba3, instconfig=singleobj_J, detconfig=mosfire_120s),
                             ObservingBlock(target=t2, pattern=abba3, instconfig=singleobj_Y, detconfig=mosfire_180s),
                            ])
myOBs3

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      
---------------|----------------------|---------------------------------------------|------------------------------------
M42            |SkyStar (10 10) x1    |longslit_align_46x0.7 J-imaging              |[MOSFIRE 10s (CDS, 9 coadds) x1]    
M42            |ABBA (1.25 arcsec) x3 |longslit_46x0.7 J-spectroscopy               |[MOSFIRE 120s (MCDS16, 1 coadds) x1]
M42            |ABBA (1.25 arcsec) x3 |longslit_46x0.7 Y-spectroscopy               |[MOSFIRE 180s (MCDS16, 1 coadds) x1]